In [95]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../../src/")
sys.path.insert(0, "../../../projects/skimpy_bench/src/")
sys.path.insert(0, "../../../projects/skimpy_blox/")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
import random
import numpy as np
import skimpy
import skimpy_3d
from skimpy_blox import minecraft, colors
from skimpy import functional as F
import ipywebrtc

SHOW_RUNS_MODE = False

In [97]:
def show_runs(t, runs_value=2):
    q = t.reshape(len(t))
    ret = skimpy.Tensor(len(t), val=q.to(bool).to(int)._tensor.array())
    run_points = runs_value * (
        (
            (q[:(len(q) - 1)] == q[1:])
        )
        & (q[1:] != 0)
    ).to(int)

    ret[1:] = ret[1:].max(run_points)
    ret[0:(len(ret) - 1)] = ret[0:(len(ret) - 1)].max(runs_value * (ret[1:] == 2).to(int))
    return ret.reshape(t.shape)
    

In [98]:
%%time
# level = minecraft.SkimpyMinecraftLevel.load("../data/sample_minecraft_world.pickle.gz")
level = minecraft.SkimpyMinecraftLevel.load("../data/Hogwarts1.0.pickle.gz")

CPU times: user 587 ms, sys: 0 ns, total: 587 ms
Wall time: 586 ms


In [99]:
%%time
t = level.megatensor()
# t = level.chunk_list[0].tensor
if SHOW_RUNS_MODE:
    t = show_runs(t)

CPU times: user 27.3 s, sys: 347 ms, total: 27.7 s
Wall time: 3.13 s


In [100]:
for i in range(20):
    print(f"{i}=>{F.sum((t == i).to(int))}")

0=>669083423
1=>1070920
2=>585086
3=>2185407
4=>2095
5=>26584
6=>0
7=>861692
8=>0
9=>7172236
10=>0
11=>0
12=>1429
13=>1122
14=>0
15=>0
16=>0
17=>28831
18=>327296
19=>0


In [101]:
config = skimpy_3d.VoxelConfig()
config[0] = skimpy_3d.EmptyVoxel()
if SHOW_RUNS_MODE:
    config[1] = skimpy_3d.ColorVoxel(255, 255, 255)
    config[2] = skimpy_3d.ColorVoxel(255, 105, 180)
else:    
    for i in range(1, 256):
        r, g, b = colors.color_for_id(i)
        config[i] = skimpy_3d.ColorVoxel(r, g, b)

In [102]:
%%time
mesh = skimpy_3d.generate_mesh(config, t._tensor)

CPU times: user 5.2 s, sys: 480 ms, total: 5.68 s
Wall time: 1.62 s


29.843810116358068

In [104]:
from pythreejs import *
import numpy as np
from IPython.display import display

In [105]:
%%time 

geometry = BufferGeometry(
    attributes={
        "position": BufferAttribute(
            np.array(mesh.positions, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "normal": BufferAttribute(
            np.array(mesh.normals, dtype='float32').reshape(-1, 3),
            normalized=False,
        ),
        "color": BufferAttribute(
            np.array(mesh.colors, dtype='uint8').reshape(-1, 4)
        ),
        "index": BufferAttribute(
            np.array(mesh.triangles, dtype='uint32'),
            normalized=False,
        ),
    },
)

CPU times: user 14 s, sys: 1.89 s, total: 15.9 s
Wall time: 15.6 s


In [106]:
js_mesh = Mesh(
    geometry=geometry,
    material=MeshLambertMaterial(vertexColors="VertexColors"),
    position=[0, 0, 0],
)
js_mesh

Preview(child=Mesh(geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(19016184, 3), dtype…

In [107]:
dir(camera)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_notifiers',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_cross_validation_lock',
 '_default_keys',
 '_display_callbacks',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_handle_custom_msg',
 '_handle_displayed',
 '_handle_msg',
 '_holding_sync',
 '_ipython_display_',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_model_id',
 '_model_module',
 '_model_module_version',
 '_model_name',
 '_msg_callbacks',
 '_notify_trait',
 '_on_potential_ret_val',
 '_on_ret_val',
 '_previewable',
 '_property_lock',
 '_register_validator',
 '_remove_

In [108]:
camera.position

(-527.2803904980559, 150.65674673055176, 592.8745949932229)

In [109]:
"""
camera_pos = level.xyz_to_skimpy_col(level.dense_dimensions())
camera = PerspectiveCamera(position=camera_pos, lookAt=(0,0,0), fov=20)
up = [0, 0, 0]
up[level.column_order[1]] = 1
camera.up = tuple(up)
"""

'\ncamera_pos = level.xyz_to_skimpy_col(level.dense_dimensions())\ncamera = PerspectiveCamera(position=camera_pos, lookAt=(0,0,0), fov=20)\nup = [0, 0, 0]\nup[level.column_order[1]] = 1\ncamera.up = tuple(up)\n'

In [110]:
light_position = level.xyz_to_skimpy_col(tuple(e / 2 for e in level.dense_dimensions()))
light_position = level.xyz_to_skimpy_col((0, 256, 0))
point_light = PointLight(color='#ffffff', position=light_position)
global_light = AmbientLight(color='#333333')

#normals_helper = VertexNormalsHelper(js_mesh, size=0.5, color="#ffcc00")

scene = Scene(children=[js_mesh, camera, point_light, global_light], background="black")

In [111]:
renderer = Renderer(
    camera=camera, 
    background_opacity=1,
    scene=scene,
    #controls=[OrbitControls(controlling=camera)],
    width=800,
    height=600,
)

display(renderer)

Renderer(camera=PerspectiveCamera(fov=20.0, position=(-527.2803904980559, 150.65674673055176, 592.874594993222…

In [112]:
renderer.scene = scene

In [113]:
stream = ipywebrtc.WidgetStream(widget=renderer, max_fps=1)
recorder = ipywebrtc.ImageRecorder(stream=stream)
recorder
#recorder.save("/home/tdimson/Downloads/render.png")


ImageRecorder(image=Image(value=b''), stream=WidgetStream(max_fps=1, widget=Renderer(camera=PerspectiveCamera(…